In [64]:
#Request test block
import requests
from bs4 import BeautifulSoup

url = "https://twitter.com/search?q=depressed&src=typed_query"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

print(soup.find_all('p')[0].get_text())

#This is what we get if perform a GET request on Twitter URL : https://twitter.com/search?q=depressed&src=typed_query

We've detected that JavaScript is disabled in your browser. Would you like to proceed to legacy Twitter?


In [21]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd
import datetime

In [51]:
#This is experiment block
date1 = datetime.datetime.date(datetime.datetime.strptime("2020-01-01", '%Y-%m-%d'))
date2 = datetime.datetime.date(datetime.datetime.strptime("2020-09-19", '%Y-%m-%d'))

days = abs(date1-date2).days
print(days//7)
print(date1 < date2)
print("hey",str(date2 - datetime.timedelta(days=7)))

txt = "https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-19&src=typed_query"
part = txt.split('%3A')
part[2] = 'newwwwwwwwwwwwww&src=typed_query'
txt = '%3A'.join(part)
print(txt)

print(datetime.datetime.date(datetime.datetime.strptime("Sep 11", "%b %d")))

37
True
hey 2020-09-12
https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3Anewwwwwwwwwwwwww&src=typed_query
1900-09-11


# Search Words/Hashtags, Input Dates

In [65]:
searchWord = "#depressed"
searchFrom = "2020-01-01"
searchUntil = "2020-09-19"

In [66]:
class PostsFromTwitter:

    def __init__(self, query, fromDate, toDate):
        
        ''' URL examples
        https://mobile.twitter.com/search?q=depressed%2Canxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23depressed%23anxiety&src=typed_query
        https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-19&src=typed_query
        '''
        
        self.query = quote(query.strip())
        self.fromDate = fromDate
        self.toDate = toDate
        self.numWeeks = self.getNumWeeks()
        
        self.after_part = (
            f"&src=typed_query"
        )
        self.url = f"https://mobile.twitter.com/search?q={self.query}+since%3A{self.fromDate}+until%3A{self.toDate}"
        self.url += self.after_part
        self.tweets = []
    
    def getNumWeeks(self):        #This function returns the number of the weeks between 2 dates.        
        
        self.dateFormat1 = datetime.datetime.date(datetime.datetime.strptime(self.fromDate, '%Y-%m-%d'))
        self.dateFormat2 = datetime.datetime.date(datetime.datetime.strptime(self.toDate, '%Y-%m-%d'))

        return (abs(date1-date2).days)//7

    def scrape_tweets(self):
        
        while self.dateFormat2 > self.dateFormat1 :
            
            self.url = f"https://mobile.twitter.com/search?q=%23depressed+since%3A{self.fromDate}+until%3A{str(self.dateFormat2)}&src=typed_query"
            
            print(f"No. of weeks to traverse : {self.numWeeks} and new URL  : {self.url}")
            
            reloadCount = 40
            while reloadCount > 0 :
                #print("URL to be scraped :: ", self.url)
                content = requests.get(self.url)
                soup = BeautifulSoup(content.text, "html.parser")
                tweet_divs = soup.select("#main_content")[0].select(".tweet")
                #print(tweet_divs)

                for tweet in tweet_divs:
                    handle = tweet.find("div", {"class": "username"}).text.replace("\n", " ").strip()
                    post = tweet.find("div", {"class": "tweet-text"}).text.replace("\n", " ").strip()
                    tweetId = tweet.find("div", {"class": "tweet-text"})['data-id']
                    timestamp = tweet.find("td", {"class": "timestamp"}).find('a').text.replace("\n", " ").strip()
                    self.tweets.append([tweetId,handle,post,timestamp])


                loadMoreButton = soup.select("#main_content")[0].select(".w-button-more")
                try:
                    nextCursorUrl = "https://mobile.twitter.com"+loadMoreButton[0].find('a')['href']
                except IndexError:
                    print("Oops!  list index out of range " , reloadCount)
                    break
                self.url = nextCursorUrl
                reloadCount = reloadCount - 1
            
            self.dateFormat2 = (self.dateFormat2 - datetime.timedelta(days=7)) #1 week less
            self.numWeeks -= 1
            
            
        return self.tweets


### Create an object of PostsFromTwitter usign parameters defined above and call the method scrape_tweets()

In [67]:
scraperObject = PostsFromTwitter(searchWord,searchFrom,searchUntil)
tweetsList = scraperObject.scrape_tweets()

No. of weeks to traverse : 37 and new URL  : https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-19&src=typed_query
Oops!  list index out of range  11
No. of weeks to traverse : 36 and new URL  : https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-12&src=typed_query
Oops!  list index out of range  32
No. of weeks to traverse : 35 and new URL  : https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-09-05&src=typed_query
Oops!  list index out of range  35
No. of weeks to traverse : 34 and new URL  : https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-08-29&src=typed_query
Oops!  list index out of range  35
No. of weeks to traverse : 33 and new URL  : https://mobile.twitter.com/search?q=%23depressed+since%3A2020-01-01+until%3A2020-08-22&src=typed_query
Oops!  list index out of range  35
No. of weeks to traverse : 32 and new URL  : https://mobile.twitter.com/search?q=%23d

In [68]:
print(tweetsList[len(tweetsList)-1])
len(tweetsList)

['1213182496407478272', '@Glass1ncision', "Got extremely discouraged in town and responded to the weather, negatively. All my plans fell apart, each one; one by one they crumbled and died. Pretty much, dead like my dreams. Just #depressed about the entire day, overall. Think it's just the fact it's so dark & dreary.", 'Jan 3']


3902

In [69]:
df = pd.DataFrame(tweetsList,columns=['tweet_id','@','tweet','date'])
df

,tweet_id,@,tweet,date
0,1306491288850706433,@Austin7397,It hurts. It hurts a lot. But I'll keep it to ...,Sep 17
1,1306827821625552897,@InveigleMagazi1,How To Cope with Depression inveiglemagazine.c...,Sep 17
2,1307095401883893764,@Zee_Kamboh_25,#Depressed...? Earphones..! Volume up..! And i...,Sep 18
3,1306960154773004289,@afh_una,"Today, once again I look empty. #depression #...",Sep 18
4,1306760408745811968,@PHMIWTKM,Some times you just gotta walk away #hurt #dep...,Sep 17
...,...,...,...,...
3897,1212649006544949248,@JosephBlade,It's 3 AM. I got 2 choices. Be #depressed that...,Jan 2
3898,1212625708612087808,@anniesdreaming,Anyone else feeling trapped and intensely emot...,Jan 1
3899,1212384194460078080,@samanthajoule,"We have a fresh new day, a fresh new year & a ...",Jan 1
3900,1213200059933822978,@msgcruz,"thats it, i am turning to depression and heavy...",Jan 3


In [70]:
df.to_csv(r'tweetsList.csv',index=False)